### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um_mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

aff = np.array([[ 1.0, 0, 0],
                [ 0, 1.0, 0],
                [ 0, 0, 1.0]], dtype='float')
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/P185/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,4",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/P185/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/P185/ants/F2M_,/flash/ReiterU/tmp/P185/ants/Warped.nii.gz,/flash/ReiterU/tmp/P185/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/P185/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P185/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/P185/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P185/ants/536BP_10um_masked.nii,1,4]', '--convergence', '[500x500x100x30,1e-6,10]', '--shrink-factors', '8x4x2x1', '--smoothing-sigmas', '3x2x1x0vox']


 2DIAGNOSTIC,     5, -1.480593379534e-01, 1.797693134862e+308, 2.6553e+00, 5.2042e-02,
 2DIAGNOSTIC,     6, -1.483217840936e-01, 1.797693134862e+308, 2.7131e+00, 5.7786e-02,
 2DIAGNOSTIC,     7, -1.488021337984e-01, 1.797693134862e+308, 2.7640e+00, 5.0878e-02,
 2DIAGNOSTIC,     8, -1.501804021603e-01, 1.797693134862e+308, 2.8173e+00, 5.3317e-02,
 2DIAGNOSTIC,     9, -1.506126656218e-01, 1.797693134862e+308, 2.8694e+00, 5.2106e-02,
 2DIAGNOSTIC,    10, -1.522054234924e-01, 1.797693134862e+308, 2.9230e+00, 5.3594e-02,
 2DIAGNOSTIC,    11, -1.608074256216e-01, 1.797693134862e+308, 2.9739e+00, 5.0897e-02,
 2DIAGNOSTIC,    12, -1.663635671079e-01, 1.797693134862e+308, 3.0259e+00, 5.1953e-02,
 2DIAGNOSTIC,    13, -1.749445238469e-01, 1.797693134862e+308, 3.0809e+00, 5.5059e-02,
 2DIAGNOSTIC,    14, -2.033402194555e-01, 1.797693134862e+308, 3.1339e+00, 5.2920e-02,
 2DIAGNOSTIC,    15, -2.639449041457e-01, 1.670276249795e-02, 3.1914e+00, 5.7513e-02,
 2DIAGNOSTIC,    16, -3.245068676197e-01, 2.

 2DIAGNOSTIC,    40, -3.610106161484e-01, 1.653370524783e-03, 2.2258e+01, 3.7345e-01,
 2DIAGNOSTIC,    41, -3.611114251628e-01, 1.601927859822e-03, 2.2593e+01, 3.3500e-01,
 2DIAGNOSTIC,    42, -3.612393610174e-01, 1.485183107752e-03, 2.2957e+01, 3.6440e-01,
 2DIAGNOSTIC,    43, -3.613417132432e-01, 1.315158060765e-03, 2.3310e+01, 3.5288e-01,
 2DIAGNOSTIC,    44, -3.617072360644e-01, 1.105044671091e-03, 2.3674e+01, 3.6430e-01,
 2DIAGNOSTIC,    45, -3.626507462443e-01, 8.746015677496e-04, 2.4004e+01, 3.3009e-01,
 2DIAGNOSTIC,    46, -3.659350903303e-01, 6.616755540436e-04, 2.4366e+01, 3.6171e-01,
 2DIAGNOSTIC,    47, -3.924657341202e-01, 7.595831962588e-04, 2.4741e+01, 3.7491e-01,
 2DIAGNOSTIC,    48, -4.063645943567e-01, 1.024051255973e-03, 2.5081e+01, 3.4040e-01,
 2DIAGNOSTIC,    49, -4.115132031766e-01, 1.448820462388e-03, 2.5424e+01, 3.4278e-01,
 2DIAGNOSTIC,    50, -4.126472208223e-01, 1.830597571264e-03, 2.5747e+01, 3.2259e-01,
 2DIAGNOSTIC,    51, -4.138011207837e-01, 2.1425562114

 2DIAGNOSTIC,    18, -4.158648514580e-01, 2.334382935837e-05, 8.1778e+01, 1.2996e+00,
 2DIAGNOSTIC,    19, -4.158660814383e-01, 1.885968557896e-05, 8.3159e+01, 1.3812e+00,
 2DIAGNOSTIC,    20, -4.158692246014e-01, 1.505323346025e-05, 8.4660e+01, 1.5011e+00,
 2DIAGNOSTIC,    21, -4.158815908026e-01, 1.237987300427e-05, 8.6309e+01, 1.6488e+00,
 2DIAGNOSTIC,    22, -4.159415211395e-01, 1.174703680856e-05, 8.7859e+01, 1.5499e+00,
 2DIAGNOSTIC,    23, -4.159601118763e-01, 1.190230736264e-05, 8.9517e+01, 1.6581e+00,
 2DIAGNOSTIC,    24, -4.159933280398e-01, 1.279562194872e-05, 9.1667e+01, 2.1494e+00,
 2DIAGNOSTIC,    25, -4.160369920492e-01, 1.408913896001e-05, 9.3356e+01, 1.6893e+00,
 2DIAGNOSTIC,    26, -4.160376821553e-01, 1.472150000125e-05, 9.5192e+01, 1.8360e+00,
 2DIAGNOSTIC,    27, -4.160602613613e-01, 1.519964226694e-05, 9.6681e+01, 1.4890e+00,
 2DIAGNOSTIC,    28, -4.160804645189e-01, 1.548117053981e-05, 9.8134e+01, 1.4530e+00,
 2DIAGNOSTIC,    29, -4.161481775606e-01, 1.6396049337

 1DIAGNOSTIC,    47, -8.385739980758e-01, 8.284815688293e-05, 5.4555e+01, 1.0965e+00,
 1DIAGNOSTIC,    48, -8.391302942489e-01, 8.071728425033e-05, 5.5661e+01, 1.1052e+00,
 1DIAGNOSTIC,    49, -8.396971028529e-01, 7.879800186383e-05, 5.6798e+01, 1.1376e+00,
 1DIAGNOSTIC,    50, -8.402777008444e-01, 7.735216670875e-05, 5.7944e+01, 1.1458e+00,
 1DIAGNOSTIC,    51, -8.408824119030e-01, 7.660257665475e-05, 5.9060e+01, 1.1159e+00,
 1DIAGNOSTIC,    52, -8.415155014180e-01, 7.655204405242e-05, 6.0155e+01, 1.0950e+00,
 1DIAGNOSTIC,    53, -8.422031226334e-01, 7.716589236454e-05, 6.1264e+01, 1.1095e+00,
 1DIAGNOSTIC,    54, -8.429154603925e-01, 7.825482731860e-05, 6.2373e+01, 1.1088e+00,
 1DIAGNOSTIC,    55, -8.436564354197e-01, 7.975158138795e-05, 6.3462e+01, 1.0884e+00,
 1DIAGNOSTIC,    56, -8.444446536416e-01, 8.176613090103e-05, 6.4577e+01, 1.1154e+00,
 1DIAGNOSTIC,    57, -8.452550101199e-01, 8.405762763478e-05, 6.5691e+01, 1.1135e+00,
 1DIAGNOSTIC,    58, -8.460840974161e-01, 8.6427309351

 1DIAGNOSTIC,   143, -8.666715323442e-01, 8.263926069121e-06, 1.6370e+02, 1.1288e+00,
 1DIAGNOSTIC,   144, -8.668375273468e-01, 8.445099818567e-06, 1.6487e+02, 1.1706e+00,
 1DIAGNOSTIC,   145, -8.670051965426e-01, 8.508863469109e-06, 1.6604e+02, 1.1769e+00,
 1DIAGNOSTIC,   146, -8.671698544010e-01, 8.445228356327e-06, 1.6717e+02, 1.1253e+00,
 1DIAGNOSTIC,   147, -8.673592143733e-01, 8.338485398770e-06, 1.6832e+02, 1.1474e+00,
 1DIAGNOSTIC,   148, -8.675646747154e-01, 8.291556541463e-06, 1.6948e+02, 1.1595e+00,
 1DIAGNOSTIC,   149, -8.677887015379e-01, 8.343359315021e-06, 1.7061e+02, 1.1344e+00,
 1DIAGNOSTIC,   150, -8.680271207630e-01, 8.556678234367e-06, 1.7177e+02, 1.1638e+00,
 1DIAGNOSTIC,   151, -8.682885203112e-01, 8.978614845168e-06, 1.7295e+02, 1.1778e+00,
 1DIAGNOSTIC,   152, -8.685523628860e-01, 9.490406084317e-06, 1.7411e+02, 1.1563e+00,
 1DIAGNOSTIC,   153, -8.688292233150e-01, 1.006181246382e-05, 1.7525e+02, 1.1404e+00,
 1DIAGNOSTIC,   154, -8.691065619665e-01, 1.0624898255

 1DIAGNOSTIC,   239, -8.859872914321e-01, 5.368092304661e-06, 2.7375e+02, 1.0955e+00,
 1DIAGNOSTIC,   240, -8.861635611778e-01, 5.245758558821e-06, 2.7475e+02, 1.0013e+00,
 1DIAGNOSTIC,   241, -8.863339370992e-01, 5.141044902152e-06, 2.7563e+02, 8.7822e-01,
 1DIAGNOSTIC,   242, -8.864990431716e-01, 5.041545405676e-06, 2.7678e+02, 1.1482e+00,
 1DIAGNOSTIC,   243, -8.866676801377e-01, 4.973106478449e-06, 2.7792e+02, 1.1353e+00,
 1DIAGNOSTIC,   244, -8.868224335866e-01, 4.874717272048e-06, 2.7905e+02, 1.1362e+00,
 1DIAGNOSTIC,   245, -8.869787949015e-01, 4.768788010459e-06, 2.8017e+02, 1.1182e+00,
 1DIAGNOSTIC,   246, -8.871288315331e-01, 4.663044191999e-06, 2.8130e+02, 1.1249e+00,
 1DIAGNOSTIC,   247, -8.872808479973e-01, 4.557692548568e-06, 2.8247e+02, 1.1732e+00,
 1DIAGNOSTIC,   248, -8.874257154715e-01, 4.438301687180e-06, 2.8354e+02, 1.0736e+00,
 1DIAGNOSTIC,   249, -8.875743477567e-01, 4.330019852812e-06, 2.8445e+02, 9.0338e-01,
 1DIAGNOSTIC,   250, -8.877205765431e-01, 4.2358479311

 1DIAGNOSTIC,   335, -8.972372675975e-01, 1.810635834691e-06, 3.8266e+02, 1.2481e+00,
 1DIAGNOSTIC,   336, -8.973198229973e-01, 1.782817654099e-06, 3.8391e+02, 1.2485e+00,
 1DIAGNOSTIC,   337, -8.973934301366e-01, 1.752779031190e-06, 3.8503e+02, 1.1251e+00,
 1DIAGNOSTIC,   338, -8.974683415938e-01, 1.731235155554e-06, 3.8616e+02, 1.1289e+00,
 1DIAGNOSTIC,   339, -8.975451364381e-01, 1.694926742119e-06, 3.8732e+02, 1.1593e+00,
 1DIAGNOSTIC,   340, -8.976064240036e-01, 1.650310863771e-06, 3.8849e+02, 1.1670e+00,
 1DIAGNOSTIC,   341, -8.976820051825e-01, 1.619316321694e-06, 3.8960e+02, 1.1143e+00,
 1DIAGNOSTIC,   342, -8.977563040307e-01, 1.579276226584e-06, 3.9074e+02, 1.1393e+00,
 1DIAGNOSTIC,   343, -8.978351491538e-01, 1.530697326559e-06, 3.9190e+02, 1.1619e+00,
 1DIAGNOSTIC,   344, -8.979089126211e-01, 1.511293766648e-06, 3.9301e+02, 1.1118e+00,
 1DIAGNOSTIC,   345, -8.979818566605e-01, 1.501532558762e-06, 3.9412e+02, 1.1065e+00,
 1DIAGNOSTIC,   346, -8.980533834174e-01, 1.4989932509

 1DIAGNOSTIC,    63, -9.073218638228e-01, 1.753891458709e-06, 9.9655e+02, 8.9866e+00,
 1DIAGNOSTIC,    64, -9.072485626995e-01, 1.477816461790e-06, 1.0059e+03, 9.3855e+00,
 1DIAGNOSTIC,    65, -9.073043038109e-01, 1.373253251641e-06, 1.0150e+03, 9.0490e+00,
 1DIAGNOSTIC,    66, -9.072963781879e-01, 1.441991824457e-06, 1.0247e+03, 9.6934e+00,
 1DIAGNOSTIC,    67, -9.073526570908e-01, 1.676959618290e-06, 1.0338e+03, 9.0989e+00,
 1DIAGNOSTIC,    68, -9.072763010714e-01, 1.389304773402e-06, 1.0431e+03, 9.3546e+00,
 1DIAGNOSTIC,    69, -9.073307779631e-01, 1.273384195005e-06, 1.0529e+03, 9.7421e+00,
 1DIAGNOSTIC,    70, -9.073260026274e-01, 1.335675181234e-06, 1.0618e+03, 8.9022e+00,
 1DIAGNOSTIC,    71, -9.073795549559e-01, 1.214677423294e-06, 1.0706e+03, 8.7870e+00,
 1DIAGNOSTIC,    72, -9.073020598127e-01, 9.372951452497e-07, 1.0786e+03, 8.0764e+00,
XXDIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 1DIAGNOSTIC,     1, -8.844998322840e-01, 1.797693134862

 1DIAGNOSTIC,    86, -9.006660586123e-01, 3.320004956094e-06, 6.2363e+03, 6.0344e+01,
 1DIAGNOSTIC,    87, -9.007241790690e-01, 3.342385701492e-06, 6.2972e+03, 6.0826e+01,
 1DIAGNOSTIC,    88, -9.007768774875e-01, 3.487123498342e-06, 6.3557e+03, 5.8524e+01,
 1DIAGNOSTIC,    89, -9.007804553682e-01, 3.389574578540e-06, 6.4171e+03, 6.1372e+01,
 1DIAGNOSTIC,    90, -9.008777990453e-01, 3.566201021606e-06, 6.4768e+03, 5.9712e+01,
 1DIAGNOSTIC,    91, -9.008753993538e-01, 3.462223744189e-06, 6.5383e+03, 6.1535e+01,
 1DIAGNOSTIC,    92, -9.009676058833e-01, 3.589945839119e-06, 6.5984e+03, 6.0122e+01,
 1DIAGNOSTIC,    93, -9.009614689662e-01, 3.430644117858e-06, 6.6588e+03, 6.0362e+01,
 1DIAGNOSTIC,    94, -9.010521191322e-01, 3.501676041353e-06, 6.7214e+03, 6.2581e+01,
 1DIAGNOSTIC,    95, -9.010410399152e-01, 3.265044212362e-06, 6.7804e+03, 5.9030e+01,
 1DIAGNOSTIC,    96, -9.011267362323e-01, 3.232086916660e-06, 6.8396e+03, 5.9185e+01,
 1DIAGNOSTIC,    97, -9.011140020001e-01, 3.0592194444

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P185/ants', 'deigo:/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/analysis'], returncode=0)